# Using Llama 3.2 1B SLM with Python & HuggingFace for diverse real-world tasks using Prompt Engineering

In this notebook you will use Meta Llama 3.2 1B SLM via HuggingFace to solve:

- Task 1: Zero-shot Classification
- Task 2: Few-shot Classification
- Task 3: Coding Tasks - Python
- Task 4: Coding Tasks - SQL
- Task 5: Information Extraction
- Task 6: Closed-Domain Question Answering
- Task 7: Open-Domain Question Answering
- Task 8: Document Summarization
- Task 9: Transformation
- Task 10: Translation



___Created By: Dipanjan (DJ)___

## Install HuggingFace dependencies

In [1]:
!pip install transformers==4.46.2
!pip install huggingface_hub==0.26.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 4.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [transformers] [transformers]
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.34.5
    Uninstalling huggingface-hub-0.34.5:
      Successfully uninstalled huggingface-hub-0.34.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.45.0 requires huggingface-hub<1.0,>=0.33.5, but you have huggingface-hub 0.26.2 which is incompatible.


## Load Hugging Face Access Token


In [ ]:
from getpass import getpass

hf_key = getpass("Enter HuggingFace API Key: ")

## Configure Key in Environment


In [3]:
import os

os.environ["HF_TOKEN"] = hf_key

## Make sure you have access to Llama 3.2 1B on HuggingFace

Certain LLMs are gated like [Meta Llama 3.2 1B Instruct](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct) so make sure to apply for access as shown below else you will get an error when using the model.

__Note: Sometimes it might take a day to get approval from Meta and if you old token still doesn't work, generate a new access token as you did before and use it.__

![](https://i.imgur.com/M88MOu5.png)

## Using LLMs via Hugging Face Inference Client

Thankfully HuggingFace has made its new [__Inference Client__](https://huggingface.co/docs/huggingface_hub/en/package_reference/inference_client) free to use with some basic rate limits etc. in place so you don't end up making unlimited requests on its servers.

The best part is you can access 150,000+ deep learning models without worrying about your infrastructure. Similar to the inference API

In [4]:
from huggingface_hub import InferenceClient

# Find more models here
# https://huggingface.co/models?pipeline_tag=text-generation&sort=trending
def get_completion(prompt, api_key=hf_key, model="meta-llama/Llama-3.2-1B-Instruct"):
    client = InferenceClient(model=model, api_key=api_key)
    chat = [
        { "role": "user", "content": prompt},
    ]
    response = client.chat_completion(chat, temperature=0, max_tokens=2048)
    return response.choices[0].message.content

## Let's try out the Llama 3.2 1B model!

In [ ]:
from IPython.display import Markdown, display

response = get_completion(prompt='Explain Generative AI in 2 bullet points')
display(Markdown(response))

- Generative AI refers to a class of artificial intelligence systems designed to create new content by learning from a dataset, simulating creativity, and generating outputs that are often indistinguishable from human-made content.

- It leverages deep learning techniques, such as neural networks, to autonomously produce complex data samples (e.g., images, music, text) by understanding and replicating patterns, structures, and styles found in the training data, enabling applications like content creation, design, and augmentation of existing datasets.

## Task 1: Zero-shot Classification

This prompt tests an LLM's text classification capabilities by prompting it to classify a piece of text without providing any examples.



In [ ]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [ ]:
responses = []

for review in reviews:
  prompt = f"""
              Act as a product review analyst.
              Given the following review,
              Display the overall sentiment for the review as only one of the following:
              Positive, Negative OR Neutral

              ```{review}```
              """
  response = get_completion(prompt)
  responses.append(response)

In [ ]:
for review, response in zip(reviews, responses):
  print('Review:', review)
  print('Sentiment:', response)
  print('------')
  print('\n')

Review: 
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    
Sentiment: The overall sentiment of the review is Positive.

The reviewer uses phrases such as "fantastic", "impressively clear", "just the right amount of bass", and "bargain" to express their satisfaction with the product. They also mention specific features, such as the waterproof design and secure packaging, which further supports the positive sentiment.
------


Review: 
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than coo

## Task 2: Few-shot Classification

This prompt tests an LLM's text classification capabilities by prompting it to classify a piece of text by providing a few examples of inputs and outputs.



In [ ]:
responses = []

for review in reviews:
  prompt = f"""
              Act as a product review analyst.
              Given the following review,
              Display only the overall sentiment for the review:

              Try to classify it by using the following examples as a reference:

              Review: Just received the Laptop I ordered for work, and it's amazing.
              Sentiment: 😊

              Review: Needed a new mechanical keyboard, but this model has been totally disappointing.
              Sentiment: 😡

              Review: ```{review}```
              """
  response = get_completion(prompt)
  responses.append(response)

In [ ]:
for review, response in zip(reviews, responses):
  print('Review:', review)
  print('Sentiment:', response)
  print('------')
  print('\n')

Review: 
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    
Sentiment: The overall sentiment of the review is: Positive
------


Review: 
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    
Sentiment: The

## Task 3: Coding Tasks - Python

This prompt tests an LLM's capabilities for generating python code based on various tasks


In [ ]:
prompt = f"""
Act as an expert in generating python code

Your task is to generate python code
to build a Chain of Though prompt pattern using
the openai python library
"""
response = get_completion(prompt)

In [ ]:
from IPython.display import Markdown, display

display(Markdown(response))

Here's an example of how you can build a Chain of Thought prompt pattern using the OpenAI Python library. This code will generate a prompt that asks a series of questions to the user, encouraging them to think deeply and critically.

```python
import random

class ChainOfThought:
    def __init__(self):
        self.questions = {
            "What is the meaning of life?": "What do you think is the most important aspect of human existence?",
            "Can you describe a time when you felt truly happy?": "What were the circumstances that led to this feeling of happiness?",
            "What is the biggest risk you've ever taken?": "What did you learn from this experience, and how has it shaped your perspective?",
            "Can you explain the concept of free will?": "How do you think our choices and actions are influenced by factors beyond our control?",
            "What is the most significant challenge you've faced in your life?": "How did you overcome this challenge, and what did you learn from the experience?"
        }

    def generate_prompt(self):
        prompt = random.choice(list(self.questions.keys()))
        return prompt

    def ask_question(self, prompt):
        print(prompt)
        answer = input("Your turn! Please respond with a question or statement that answers the prompt: ")
        return answer

    def chain_of_thought(self):
        while True:
            prompt = self.generate_prompt()
            answer = self.ask_question(prompt)
            print(f"Your turn! Please respond with a question or statement that answers the prompt: {prompt}")
            print(f"Your answer: {answer}")
            print()

# Example usage:
chain_of_thought = ChainOfThought()
chain_of_thought.chain_of_thought()
```

This code defines a `ChainOfThought` class that contains a dictionary of questions and their corresponding answers. The `generate_prompt` method selects a random prompt from the dictionary, and the `ask_question` method asks the user to respond with a question or statement that answers the prompt. The `chain_of_thought` method generates a prompt, asks the user to respond, and then repeats the process until the user chooses to stop.

You can add or remove questions and answers from the dictionary as needed. This code provides a basic structure for building a Chain of Thought prompt pattern using the OpenAI Python library.

## Task 4: Coding Tasks - SQL

This prompt tests an LLM's capabilities for generating SQL code based on various tasks

In [ ]:
prompt = f"""
Act as an expert in generating SQL code.

Understand the following schema of the database tables carefully:
Table departments, columns = [DepartmentId, DepartmentName]
Table employees, columns = [EmployeeId, EmployeeName, DepartmentId]
Table salaries, columns = [EmployeeId, Salary]

Create a MySQL query for the employee with max salary in the 'IT' Department.
"""
response = get_completion(prompt)

In [ ]:
display(Markdown(response))

To find the employee with the maximum salary in the 'IT' department, you can use the following MySQL query:

```sql
SELECT E.EmployeeName
FROM employees E
JOIN departments D ON E.DepartmentId = D.DepartmentId
WHERE D.DepartmentName = 'IT'
ORDER BY E.Salary DESC
LIMIT 1;
```

This query works as follows:

1. It joins the `employees` table with the `departments` table on the `DepartmentId` column.
2. It filters the results to only include employees in the 'IT' department.
3. It orders the results in descending order by salary (highest salary first).
4. Finally, it limits the results to a single row, which is the employee with the maximum salary in the 'IT' department.

Note: If there are multiple employees with the same maximum salary in the 'IT' department, this query will return only one of them. If you want to return all employees with the maximum salary, you can use the following query:

```sql
SELECT E.EmployeeName
FROM employees E
JOIN departments D ON E.DepartmentId = D.DepartmentId
WHERE D.DepartmentName = 'IT'
ORDER BY E.Salary DESC
LIMIT 1;
```

This query will return all employees with the maximum salary in the 'IT' department.

## Task 5: Information Extraction

This prompt tests an LLM's capabilities for extracting and analyzing key entities from documents

In [ ]:
clinical_note = """
60-year-old man in NAD with a h/o CAD, DM2, asthma, pharyngitis, SBP,
and HTN on altace for 8 years awoke from sleep around 1:00 am this morning
with a sore throat and swelling of the tongue.
He came immediately to the ED because he was having difficulty swallowing and
some trouble breathing due to obstruction caused by the swelling.
He did not have any associated SOB, chest pain, itching, or nausea.
He has not noticed any rashes.
He says that he feels like it is swollen down in his esophagus as well.
He does not recall vomiting but says he might have retched a bit.
In the ED he was given 25mg benadryl IV, 125 mg solumedrol IV,
and pepcid 20 mg IV.
Family history of CHF and esophageal cancer (father).
"""

In [ ]:
prompt = f"""
Act as an expert in analyzing and understanding clinical doctor notes in healthcare.
Extract all symptoms only from the clinical note information below.
Differentiate between symptoms that are present vs. absent.
Give me the probability (high/ medium/ low) of how sure you are about the result.
Add a note on the probabilities and why you think so.

Output as a markdown table with the following columns,
all symptoms should be expanded and no acronyms unless you don't know:

Symptoms | Present/Denies | Probability.

Also expand all acronyms.
Output that also as a separate appendix table in Markdown.

Clinical Note:
```{clinical_note}```
"""
response = get_completion(prompt)

In [ ]:
display(Markdown(response))

**Symptoms Table**

| Symptom | Present/Denies | Probability |
| --- | --- | --- |
| Sore throat | Present | High |
| Swelling of the tongue | Present | High |
| Difficulty swallowing | Present | High |
| Trouble breathing due to obstruction | Present | High |
| Swelling in the esophagus | Present | High |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies | Low |
| Retching | Present | Medium |
| SOB (Shortness of breath) | Denies | Low |
| Chest pain | Denies | Low |
| Itching | Denies | Low |
| Nausea | Denies | Low |
| Rashes | Denies | Low |
| Vomiting | Denies |

## Task 6: Closed-Domain Question Answering

Question Answering (QA) is a natural language processing task which involves generating the desired answer for the given question. Question Answering can be open-domain QA or closed-domain QA depending on whether the LLM is provided with the relevant context or not.

In the case of closed-domain QA, a question along with relevant context is given. Here the context is nothing but the relevant text which ideally should have the answer. Just like a RAG workflow.

In [ ]:
report = """
Three quarters (77%) of the population saw an increase in their regular outgoings over the past year,
according to findings from our recent consumer survey. In contrast, just over half (54%) of respondents
had an increase in their salary, which suggests that the burden of costs outweighing income remains for
most. In total, across the 2,500 people surveyed, the increase in outgoings was 18%, three times higher
than the 6% increase in income.

Despite this, the findings of our survey suggest we have reached a plateau. Looking at savings,
for example, the share of people who expect to make regular savings this year is just over 70%,
broadly similar to last year. Over half of those saving plan to use some of the funds for residential
property. A third are saving for a deposit, and a further 20% for an investment property or second home.

But for some, their plans are being pushed back. 9% of respondents stated they had planned to purchase
a new home this year but have now changed their mind. While for many the deposit may be an issue,
the other driving factor remains the cost of the mortgage, which has been steadily rising the last
few years. For those that currently own a property, the survey showed that in the last year,
the average mortgage payment has increased from £668.51 to £748.94, or 12%."""


In [ ]:
question = """
How much has the average mortage payment increased in the last year?
"""

prompt = f"""
Using the following context information below please answer the following question
to the best of your ability
Context:
{report}
Question:
{question}
Answer:
"""

In [ ]:
response = get_completion(prompt)
display(Markdown(response))

The average mortgage payment has increased from £668.51 to £748.94, which is an increase of £80.43.

In [ ]:
question = """
What percentage of people had an increase in salary last year? Show the answer just as a number.
"""

prompt = f"""
Using the following context information below please answer the following question
to the best of your ability
Context:
{report}
Question:
{question}
Answer:
"""
response = get_completion(prompt)
display(Markdown(response))

54%

## Task 7: Open-Domain Question Answering

Question Answering (QA) is a natural language processing task which involves generating the desired answer for the given question.

In the case of open-domain QA, only the question is asked without providing any context or information. Here, the LLM answers the question using the knowledge gained from large volumes of text data during its training. This is basically Zero-Shot QA

In [ ]:
prompt = f"""
Please answer the following question to the best of your ability
Question:
What is LangChain?

Answer:
"""

In [ ]:
response = get_completion(prompt)
display(Markdown(response))

LangChain is a decentralized, open-source, and community-driven project that aims to create a more secure and transparent blockchain. It is a fork of the existing Hyperledger Fabric blockchain platform, with the goal of improving its scalability, security, and usability.

LangChain is designed to be more flexible and adaptable than traditional blockchain platforms, allowing it to be used for a wide range of applications, including supply chain management, identity verification, and more. The project is led by a community of developers and researchers who are committed to making LangChain a viable and practical solution for real-world use cases.

LangChain's key features include:

1. Improved scalability: LangChain is designed to be more scalable than traditional blockchain platforms, allowing it to handle a larger number of transactions per second.
2. Enhanced security: LangChain incorporates advanced security measures, including encryption, access controls, and smart contract functionality.
3. Increased usability: LangChain is designed to be more user-friendly than traditional blockchain platforms, with a focus on simplicity and ease of use.

Overall, LangChain is an innovative project that is pushing the boundaries of what is possible with blockchain technology. Its focus on scalability, security, and usability makes it an attractive option for a wide range of use cases.

In [ ]:
prompt = f"""
Please answer the following question to the best of your ability
Question:
What is LangGraph?

Answer:
"""

In [ ]:
response = get_completion(prompt)
display(Markdown(response))

LangGraph is a graph database that provides a scalable and flexible way to store and query graph data. It is designed to handle large-scale graph processing and analysis workloads, making it suitable for applications such as social network analysis, recommendation systems, and knowledge graph construction.

LangGraph is built on top of a distributed graph database, allowing it to scale horizontally and handle high volumes of data. It provides a range of features, including graph data types, node and edge attributes, and query capabilities, making it a popular choice for graph data analysis and processing.

LangGraph is also known for its support for various graph algorithms and data structures, including graph traversals, shortest paths, and clustering. Additionally, it provides a flexible data model that allows users to define their own graph data types and attributes, making it a versatile tool for a wide range of applications.

Overall, LangGraph is a powerful and flexible graph database that is well-suited for large-scale graph processing and analysis workloads.

## Task 8: Document Summarization

Document summarization is a natural language processing task which involves creating a concise summary of the given text, while still capturing all the important information.

In [ ]:
doc = """
Coronaviruses are a large family of viruses which may cause illness in animals or humans.
In humans, several coronaviruses are known to cause respiratory infections ranging from the
common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS).
The most recently discovered coronavirus causes coronavirus disease COVID-19.
COVID-19 is the infectious disease caused by the most recently discovered coronavirus.
This new virus and disease were unknown before the outbreak began in Wuhan, China, in December 2019.
COVID-19 is now a pandemic affecting many countries globally.
The most common symptoms of COVID-19 are fever, dry cough, and tiredness.
Other symptoms that are less common and may affect some patients include aches
and pains, nasal congestion, headache, conjunctivitis, sore throat, diarrhea,
loss of taste or smell or a rash on skin or discoloration of fingers or toes.
These symptoms are usually mild and begin gradually.
Some people become infected but only have very mild symptoms.
Most people (about 80%) recover from the disease without needing hospital treatment.
Around 1 out of every 5 people who gets COVID-19 becomes seriously ill and develops difficulty breathing.
Older people, and those with underlying medical problems like high blood pressure, heart and lung problems,
diabetes, or cancer, are at higher risk of developing serious illness.
However, anyone can catch COVID-19 and become seriously ill.
People of all ages who experience fever and/or  cough associated with difficulty breathing/shortness of breath,
chest pain/pressure, or loss of speech or movement should seek medical attention immediately.
If possible, it is recommended to call the health care provider or facility first,
so the patient can be directed to the right clinic.
People can catch COVID-19 from others who have the virus.
The disease spreads primarily from person to person through small droplets from the nose or mouth,
which are expelled when a person with COVID-19 coughs, sneezes, or speaks.
These droplets are relatively heavy, do not travel far and quickly sink to the ground.
People can catch COVID-19 if they breathe in these droplets from a person infected with the virus.
This is why it is important to stay at least 1 meter) away from others.
These droplets can land on objects and surfaces around the person such as tables, doorknobs and handrails.
People can become infected by touching these objects or surfaces, then touching their eyes, nose or mouth.
This is why it is important to wash your hands regularly with soap and water or clean with alcohol-based hand rub.
Practicing hand and respiratory hygiene is important at ALL times and is the best way to protect others and yourself.
When possible maintain at least a 1 meter distance between yourself and others.
This is especially important if you are standing by someone who is coughing or sneezing.
Since some infected persons may not yet be exhibiting symptoms or their symptoms may be mild,
maintaining a physical distance with everyone is a good idea if you are in an area where COVID-19 is circulating."""

In [ ]:
prompt = f"""
You are an expert in generating accurate document summaries.
Generate a summary of the given document.

Document:
{doc}


Constraints: Please start the summary with the delimiter 'Summary'
and limit the summary to 5 lines

Summary:
"""

response = get_completion(prompt)
display(Markdown(response))

Summary:
Coronaviruses are a large family of viruses that can cause illness in both animals and humans. COVID-19, the most recently discovered coronavirus, is a severe respiratory disease that affects many countries globally. Symptoms include fever, dry cough, and tiredness, with some people experiencing mild or severe illness. The disease spreads primarily through small droplets from the nose or mouth, and proper hand hygiene is crucial to prevent infection. Maintaining a physical distance of at least 1 meter from others is also essential to protect both individuals and those around them.

## Task 9: Transformation

You can use LLMs to take an existing document and transform it into other formats of content and even generate training data for fine-tuning or training models

In [ ]:
fact_sheet_mobile = """
PRODUCT NAME
Samsung Galaxy Z Fold4 5G Black
​
PRODUCT OVERVIEW
Stands out. Stands up. Unfolds.
The Galaxy Z Fold4 does a lot in one hand with its 15.73 cm(6.2-inch) Cover Screen.
Unfolded, the 19.21 cm(7.6-inch) Main Screen lets you really get into the zone.
Pushed-back bezels and the Under Display Camera means there's more screen
and no black dot getting between you and the breathtaking Infinity Flex Display.
Do more than more with Multi View. Whether toggling between texts or catching up
on emails, take full advantage of the expansive Main Screen with Multi View.
PC-like power thanks to Qualcomm Snapdragon 8+ Gen 1 processor in your pocket,
transforms apps optimized with One UI to give you menus and more in a glance
New Taskbar for PC-like multitasking. Wipe out tasks in fewer taps. Add
apps to the Taskbar for quick navigation and bouncing between windows when
you're in the groove.4 And with App Pair, one tap launches up to three apps,
all sharing one super-productive screen
Our toughest Samsung Galaxy foldables ever. From the inside out,
Galaxy Z Fold4 is made with materials that are not only stunning,
but stand up to life's bumps and fumbles. The front and rear panels,
made with exclusive Corning Gorilla Glass Victus+, are ready to resist
sneaky scrapes and scratches. With our toughest aluminum frame made with
Armor Aluminum, this is one durable smartphone.
World’s first water resistant foldable smartphones. Be adventurous, rain
or shine. You don't have to sweat the forecast when you've got one of the
world's first water-resistant foldable smartphones.
​
PRODUCT SPECS
OS - Android 12.0
RAM - 12 GB
Product Dimensions - 15.5 x 13 x 0.6 cm; 263 Grams
Batteries - 2 Lithium Ion batteries required. (included)
Item model number - SM-F936BZKDINU_5
Wireless communication technologies - Cellular
Connectivity technologies - Bluetooth, Wi-Fi, USB, NFC
GPS - True
Special features - Fast Charging Support, Dual SIM, Wireless Charging, Built-In GPS, Water Resistant
Other display features - Wireless
Device interface - primary - Touchscreen
Resolution - 2176x1812
Other camera features - Rear, Front
Form factor - Foldable Screen
Colour - Phantom Black
Battery Power Rating - 4400
Whats in the box - SIM Tray Ejector, USB Cable
Manufacturer - Samsung India pvt Ltd
Country of Origin - China
Item Weight - 263 g
"""

In [ ]:
prompt =f"""Turn the following product description into a list of frequently asked questions (FAQ).
Show both the question and it's corresponding answer.
Create at the max 8 FAQs

Product description:
```{fact_sheet_mobile}```
"""

response = get_completion(prompt)
display(Markdown(response))

Here are the FAQs based on the product description:

**Q1: What is the display size of the Samsung Galaxy Z Fold4 5G Black?**
A1: The display size of the Samsung Galaxy Z Fold4 5G Black is 19.21 cm (7.6-inch) for the Main Screen and 15.73 cm (6.2-inch) for the Cover Screen.

**Q2: What is the resolution of the display?**
A2: The resolution of the display is 2176x1812.

**Q3: What is the battery life of the Samsung Galaxy Z Fold4 5G Black?**
A3: The battery life of the Samsung Galaxy Z Fold4 5G Black is 4400.

**Q4: What is the processor used in the Samsung Galaxy Z Fold4 5G Black?**
A4: The processor used in the Samsung Galaxy Z Fold4 5G Black is Qualcomm Snapdragon 8+ Gen 1.

**Q5: What is the camera resolution of the Samsung Galaxy Z Fold4 5G Black?**
A5: The camera resolution of the Samsung Galaxy Z Fold4 5G Black is Rear: 50MP, Front: 32MP.

**Q6: What is the durability of the Samsung Galaxy Z Fold4 5G Black?**
A6: The Samsung Galaxy Z Fold4 5G Black is made with materials that are not only stunning but also resistant to life's bumps and fumbles, including the front and rear panels made with exclusive Corning Gorilla Glass Victus+ and the toughest aluminum frame made with Armor Aluminum.

**Q7: What is the water resistance of the Samsung Galaxy Z Fold4 5G Black?**
A7: The Samsung Galaxy Z Fold4 5G Black is water resistant, with a rating of World's first water resistant foldable smartphones.

**Q8: What is the price of the Samsung Galaxy Z Fold4 5G Black?**
A8: The price of the Samsung Galaxy Z Fold4 5G Black is not specified in the provided information.

## Task 10: Translation

You can use LLMs to take an existing document and translate it from a source to target language. You can also translate to multiple languages at the same time

In [ ]:
prompt = """You are an expert translator.
Translate the given text from English to German and Spanish.

Text: 'Hello, how are you today?'
Translation:
"""

response = get_completion(prompt)
display(Markdown(response))

Here are the translations:

- English: "Hello, how are you today?"
- German: "Hallo, wie geht es dir heute?"
- Spanish: "Hola, ¿cómo estás hoy?"